<a href="https://colab.research.google.com/github/mayankraj25/LLM_HF-PROJECTS/blob/main/Test_data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import requests
from openai import OpenAI
from google.colab import drive
from google.colab import userdata
from huggingface_hub import login
from IPython.display import Markdown,display,update_display
from transformers import AutoTokenizer, AutoModelForCausalLM,TextStreamer,BitsAndBytesConfig
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
import gradio as gr

In [1]:
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install transformers==4.40.1 --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.


In [4]:
hf_token=userdata.get("HF_TOKEN")
login(hf_token,add_to_git_credential=True)

In [4]:
openai_api_key=userdata.get("OPENAI_API_KEY")
openai=OpenAI(api_key=openai_api_key)

In [5]:
if torch.cuda.is_available():
  quant_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
      bnb_4bit_quant_type="nf4"
  )
  print("BitsAndBytesConfig created successfully.")
else:
  print("CUDA is not available. BitsAndBytesConfig requires a GPU.")

BitsAndBytesConfig created successfully.


In [7]:
model_id="microsoft/phi-2"

tokenizer=AutoTokenizer.from_pretrained(model_id)
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    #quantization_config=quant_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
model.eval()
streamer=TextStreamer(tokenizer)

In [34]:
def generate_synthetic_data(topic,columns):
    prompt = (
        f"Generate only a JSON array of 10 fake records about {topic} with the following columns: {columns}. "
        f"Do not include any explanations or formatting. Output only valid JSON and NO PRAMBLE."
    )
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=500)
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # Optional: clip at last closing brace
        if "}" in decoded_output:
            decoded_output = decoded_output[:decoded_output.rfind("}")+1]

        return decoded_output
    except Exception as e:
        return f"Error: {str(e)}"

In [35]:
iface=gr.Interface(
    fn=generate_synthetic_data,
    inputs=[
        gr.Textbox(label="Topic"),
        gr.Textbox(label="Columns (Comma seperated)")
    ],
    outputs=gr.Textbox(label="Synthetic Data"),
    title="Synthetic Data Generator",
    description="Enter a topic and columns to generate a synthetic dataset.",
    allow_flagging="never"
)
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19615122c4ebbb7bf8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
